# Project 2


### Task 1
Train the CNN (as much as you can) to reach convergence. Investigate what patterns the first layer (layer 0) filters pick up by plotting the filters as small 2d images. To plot a 2d array x as image, use "imshow(x, cmap=cm.gray)". You should plot the 10 filters together using subplot. 

### Task 2
The given CNN has 2 conv&pool layers, 1 hidden layer and 1 output layer. 
Modify the CNN to have:
  - 1 conv&pool layer, 1 hidden layer and 1 output layer;
  - 1 hidden layer and 1 output layer; 
  
while keeping the other parameters the same. Compare the error rates on the test data for the original CNN and the two modifications and determine whether the conv&pool layers play a significant role for performance.

### Task 3
Change the number of filters for the two conv&pool layers:
  - try 10 filters for layer 1 and 20 for layer 2;
  - try 20 filters for layer 1 and 10 for layer 2.

Compare error rate of the two cases and that of the original. Comment on how number of filters can impact performance. 

In [7]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
import cPickle, gzip

import numpy
import time

import theano
import theano.tensor as T
from theano.tensor.signal import downsample
from theano.tensor.nnet import conv

ImportError: cannot import name downsample

In [9]:
class HiddenLayer(object):

    def __init__(self, input, n_in, n_out):
        self.input = input

        W_values = 4*numpy.random.uniform(
                low=-numpy.sqrt(6. / (n_in + n_out)),
                high=numpy.sqrt(6. / (n_in + n_out)),
                size=(n_in, n_out)
        )
        self.W = theano.shared(value=W_values, name='W', borrow=True)
        self.b = theano.shared(value=numpy.zeros((n_out,)), name='b', borrow=True)
        self.params = [self.W, self.b]

        net = T.dot(self.input, self.W) + self.b
        self.output = T.nnet.sigmoid(net)

        
class MultiLogisticRegression(object):

    def __init__(self, input, n_in, n_out):

        self.input = input

        self.W = theano.shared(
            value=numpy.zeros((n_in, n_out)),
            name='W',
            borrow=True
        )
        self.b = theano.shared(
            value = numpy.zeros((n_out,)),
            name='b',
            borrow=True
        )
        self.params = [self.W, self.b]

        self.prob = T.nnet.softmax(T.dot(self.input, self.W) + self.b)
        self.predict = T.argmax(self.prob, axis=1)

    def nll(self, y):
        return  -T.mean(T.log(self.prob)[T.arange(y.shape[0]), y])


In [10]:
class ConvPoolLayer(object):

    def __init__(self, input, filter_shape, image_shape, poolsize):

        self.input = input

        n_in = filter_shape[1]*filter_shape[2]*filter_shape[3]
        n_out = (filter_shape[0]*filter_shape[2]*filter_shape[3])/(poolsize[0]*poolsize[1])
        W_bound = numpy.sqrt(6./(n_in + n_out))
        self.W = theano.shared(
            numpy.random.uniform(low=-W_bound, high=W_bound, size=filter_shape),
            borrow=True
        )
        self.b = theano.shared(value=numpy.zeros((filter_shape[0],)), borrow=True)
        self.params = [self.W, self.b]
        
        conv_out = conv.conv2d(
            input=self.input,
            filters=self.W,
            filter_shape=filter_shape,
            image_shape=image_shape
        )

        pooled_out = downsample.max_pool_2d(
            input=conv_out,
            ds=poolsize,
            ignore_border=True
        )

        self.output = T.tanh(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))


In [11]:
def ConvNN_Origin(num_epochs):
    batch_size = 250
    learning_rate=0.1
    nkerns=[10, 10]


    x = T.matrix('x')
    y = T.ivector('y')


    layer0_input = x.reshape((batch_size, 1, 28, 28))
    layer0 = ConvPoolLayer(
        input=layer0_input,
        image_shape=(batch_size, 1, 28, 28),
        filter_shape=(nkerns[0], 1, 5, 5),
        poolsize=(2, 2)
    )

    layer1 = ConvPoolLayer(
        input=layer0.output,
        image_shape=(batch_size, nkerns[0], 12, 12),
        filter_shape=(nkerns[1], nkerns[0], 5, 5),
        poolsize=(2, 2)
    )  
    layer1_output = layer1.output.flatten(2)

    layer2 = HiddenLayer(
        input=layer1_output,
        n_in=nkerns[1]*4*4,
        n_out=50,
    )

    layer3 = MultiLogisticRegression(input=layer2.output, n_in=50, n_out=10)

    cost = layer3.nll(y)


    model_predict = theano.function(
        [x],
        layer3.predict
    )


    params = layer3.params + layer2.params + layer1.params + layer0.params
    grads = T.grad(cost, params)
    updates = [
        (param_i, param_i - learning_rate * grad_i)
        for param_i, grad_i in zip(params, grads)
    ] 

    train_model = theano.function(
        [x, y],
        cost,
        updates=updates
    )
    
    dataset = 'digits.pkl.gz' 
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()
    train_set_x, train_set_y = train_set
    test_set_x, test_set_y = test_set
    train_set_y = train_set_y.astype(numpy.int32)

    ix = []
    for i in range(10):
        ix.append(numpy.nonzero(train_set_y == i)[0][:500])
    ix = numpy.concatenate(ix)
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]
    ix = numpy.random.permutation(train_set_x.shape[0])
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]

    n_batches = train_set_x.shape[0]
    n_batches /= batch_size
    
    n_epochs = num_epochs
    c = numpy.zeros((n_epochs,))
    for i in range(n_epochs): 
        err = 0
        for b in range(n_batches):
            train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
            err += train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
        print 'iteration:', i, ', nll =', err
        c[i] = err
  
    n_testbatches = test_set_x.shape[0] / batch_size
    err = 0
    for b in range(n_testbatches):
        yp = model_predict(test_set_x[b*batch_size:(b+1)*batch_size])
        yy = test_set_y[b*batch_size:(b+1)*batch_size]
        err += len(np.nonzero(yp - yy)[0])
    err=1.0*err/len(test_set_y)
    
    
    return layer0.W,err ##just make the Making ConvNN function returns the filters and error rate

# Task I

In [12]:
#########Task I################
res=ConvNN_Origin(400) ## just make the Making ConvNN function return the filters
filters=res[0]
plt.figure(1)
for i in range(0,10):
    plt.subplot(2,5,i), imshow(filters.get_value()[i][0],cmap=cm.gray)
plt.show()

NameError: global name 'conv' is not defined

# Task 2

In [13]:
#########Task II################
def ConvNN_3_Layers(num_epochs):
    batch_size = 250
    learning_rate=0.1
    nkerns=[10,10]


    x = T.matrix('x')
    y = T.ivector('y')

    layer0_input = x.reshape((batch_size, 1, 28, 28))
    layer0 = ConvPoolLayer(
        input=layer0_input,
        image_shape=(batch_size, 1, 28, 28),
        filter_shape=(nkerns[0], 1, 5, 5),
        poolsize=(2, 2)
    )

    layer0_output = layer0.output.flatten(2)
    
    layer2 = HiddenLayer(
        input=layer0_output,
        n_in=nkerns[1]*12*12,
        n_out=50,
    )

    layer3 = MultiLogisticRegression(input=layer2.output, n_in=50, n_out=10)

    cost = layer3.nll(y)


    model_predict = theano.function(
        [x],
        layer3.predict
    )

    params = layer3.params + layer2.params + layer0.params
    grads = T.grad(cost, params)
    updates = [
        (param_i, param_i - learning_rate * grad_i)
        for param_i, grad_i in zip(params, grads)
    ] 

    train_model = theano.function(
        [x, y],
        cost,
        updates=updates
    )
    
    dataset = 'digits.pkl.gz' 
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()
    train_set_x, train_set_y = train_set
    test_set_x, test_set_y = test_set
    train_set_y = train_set_y.astype(numpy.int32)

    ix = []
    for i in range(10):
        ix.append(numpy.nonzero(train_set_y == i)[0][:500])
    ix = numpy.concatenate(ix)
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]
    ix = numpy.random.permutation(train_set_x.shape[0])
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]

    n_batches = train_set_x.shape[0]
    n_batches /= batch_size
    
    n_epochs = num_epochs
    c = numpy.zeros((n_epochs,))
    for i in range(n_epochs): 
        err = 0
        for b in range(n_batches):
            train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
            err += train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
        print 'iteration:', i, ', nll =', err
        c[i] = err
  
    n_testbatches = test_set_x.shape[0] / batch_size
    err = 0
    for b in range(n_testbatches):
        yp = model_predict(test_set_x[b*batch_size:(b+1)*batch_size])
        yy = test_set_y[b*batch_size:(b+1)*batch_size]
        err += len(np.nonzero(yp - yy)[0])
    return 1.0*err/len(test_set_y)

In [14]:
def ConvNN_2_Layers(num_epochs):
    batch_size = 250
    learning_rate=0.1
    nkerns=[10,10]


    x = T.matrix('x')
    y = T.ivector('y')

 
    layer2_input = x.reshape((batch_size, 1, 28, 28)).flatten(2)
    
    layer2 = HiddenLayer(
        input=layer2_input,
        n_in=28*28,
        n_out=50,
    )

    layer3 = MultiLogisticRegression(input=layer2.output, n_in=50, n_out=10)

    cost = layer3.nll(y)


    model_predict = theano.function(
        [x],
        layer3.predict
    )

    params = layer3.params + layer2.params 
    grads = T.grad(cost, params)
    updates = [
        (param_i, param_i - learning_rate * grad_i)
        for param_i, grad_i in zip(params, grads)
    ] 

    train_model = theano.function(
        [x, y],
        cost,
        updates=updates
    )
    
    dataset = 'digits.pkl.gz' 
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()
    train_set_x, train_set_y = train_set
    test_set_x, test_set_y = test_set
    train_set_y = train_set_y.astype(numpy.int32)

    ix = []
    for i in range(10):
        ix.append(numpy.nonzero(train_set_y == i)[0][:500])
    ix = numpy.concatenate(ix)
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]
    ix = numpy.random.permutation(train_set_x.shape[0])
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]

    n_batches = train_set_x.shape[0]
    n_batches /= batch_size
    
    n_epochs = num_epochs
    c = numpy.zeros((n_epochs,))
    for i in range(n_epochs): 
        err = 0
        for b in range(n_batches):
            train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
            err += train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
        print 'iteration:', i, ', nll =', err
        c[i] = err
  
    n_testbatches = test_set_x.shape[0] / batch_size
    err = 0
    for b in range(n_testbatches):
        yp = model_predict(test_set_x[b*batch_size:(b+1)*batch_size])
        yy = test_set_y[b*batch_size:(b+1)*batch_size]
        err += len(np.nonzero(yp - yy)[0])
    return 1.0*err/len(test_set_y)

In [15]:
## Original CNN
time_begin=time.time()
res=ConvNN_Origin(400) 
time_end=time.time()
err=res[1]
nminutes=(time_end-time_begin)/60
print "The error rate of the original CNN is "+str(err)
print "The time used for the original CNN is "+str(nminutes)+" minutes"

##1 con&pool layer, 1 hidden layer and 1 output layer
time_begin=time.time()
err=ConvNN_3_Layers(400)
time_end=time.time()
nminutes=(time_end-time_begin)/60
print "The error rate of 3 layers: "+str(err)
print "The time used for 3 layers: "+str(nminutes)+" minutes"

##1 hidden layer and 1 output layer 
time_begin=time.time()
err=ConvNN_2_Layers(400)
time_end=time.time()
nminutes=(time_end-time_begin)/60
print "The error rate of 2 layers: "+str(err)
print "The time used for 2 layers: "+str(nminutes)+" minutes"

NameError: global name 'conv' is not defined

The error rates of these three cases are comparable. However, it is noticable that the performance is much better with the conv&pool layers.

# Task 3

In [16]:
def ConvNN_Diff_Filters(num_epochs,filter_num0,filter_num1):
    batch_size = 250
    learning_rate=0.1
    nkerns=[filter_num0, filter_num1]


    x = T.matrix('x')
    y = T.ivector('y')


    layer0_input = x.reshape((batch_size, 1, 28, 28))
    layer0 = ConvPoolLayer(
        input=layer0_input,
        image_shape=(batch_size, 1, 28, 28),
        filter_shape=(nkerns[0], 1, 5, 5),
        poolsize=(2, 2)
    )

    layer1 = ConvPoolLayer(
        input=layer0.output,
        image_shape=(batch_size, nkerns[0], 12, 12),
        filter_shape=(nkerns[1], nkerns[0], 5, 5),
        poolsize=(2, 2)
    )  
    layer1_output = layer1.output.flatten(2)

    layer2 = HiddenLayer(
        input=layer1_output,
        n_in=nkerns[1]*4*4,
        n_out=50,
    )

    layer3 = MultiLogisticRegression(input=layer2.output, n_in=50, n_out=10)

    cost = layer3.nll(y)


    model_predict = theano.function(
        [x],
        layer3.predict
    )


    params = layer3.params + layer2.params + layer1.params + layer0.params
    grads = T.grad(cost, params)
    updates = [
        (param_i, param_i - learning_rate * grad_i)
        for param_i, grad_i in zip(params, grads)
    ] 

    train_model = theano.function(
        [x, y],
        cost,
        updates=updates
    )
    
    dataset = 'digits.pkl.gz' 
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()
    train_set_x, train_set_y = train_set
    test_set_x, test_set_y = test_set
    train_set_y = train_set_y.astype(numpy.int32)

    ix = []
    for i in range(10):
        ix.append(numpy.nonzero(train_set_y == i)[0][:500])
    ix = numpy.concatenate(ix)
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]
    ix = numpy.random.permutation(train_set_x.shape[0])
    train_set_x = train_set_x[ix]
    train_set_y = train_set_y[ix]

    n_batches = train_set_x.shape[0]
    n_batches /= batch_size
    
    n_epochs = num_epochs
    c = numpy.zeros((n_epochs,))
    for i in range(n_epochs): 
        err = 0
        for b in range(n_batches):
            train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
            err += train_model(train_set_x[b*batch_size:(b+1)*batch_size], train_set_y[b*batch_size:(b+1)*batch_size])
        print 'iteration:', i, ', nll =', err
        c[i] = err
  
    n_testbatches = test_set_x.shape[0] / batch_size
    err = 0
    for b in range(n_testbatches):
        yp = model_predict(test_set_x[b*batch_size:(b+1)*batch_size])
        yy = test_set_y[b*batch_size:(b+1)*batch_size]
        err += len(np.nonzero(yp - yy)[0])
    return 1.0*err/len(test_set_y)

In [17]:
#10 filters for layer1 and 20 filters for layer2
time_begin=time.time()
err=ConvNN_Diff_Filters(400,10,20)
time_end=time.time()
nminutes=(time_end-time_begin)/60
print "The error rate of 10 filters for layer1 and 20 filters for layer2: "+str(err)
print "The time used for 10 filters for layer1 and 20 filters for layer2: "+str(nminutes)+" minutes"

#20 filters for layer1 and 10 filters for layer2
time_begin=time.time()
ConvNN_Diff_Filters(400,20,10)
time_end=time.time()
nminutes=(time_end-time_begin)/60
print "The error rate of 20 filters for layer1 and 10 filters for layer2: "+str(err)
print "The time used for 20 filters for layer1 and 10 filters for layer2: "+str(nminutes)+" minutes"

NameError: global name 'conv' is not defined

With the iteration number 1200, the error rates of these two cases are the same as that of the original. However, more filters will leader to significantly worse performance. 